<a href="https://colab.research.google.com/github/karandomguy/email-2-faq/blob/experiments/Data_preprocessing(email2faq).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
import numpy as np
import pandas as pd 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
cv = CountVectorizer()
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import tokenize
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install zipfile36

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:


# import required modules
import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('question-pairs-dataset.zip',compression='zip')
 
# display dataset
print(df.head())

   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  


In [ ]:
df.head()

In [ ]:
df["question1"]=df["question1"].str.lower()
df["question2"]=df["question2"].str.lower()

In [ ]:
df.head()

In [ ]:

len(df)

In [ ]:
def cleaning(text):
    text = str(text)
    text = text.lower()
    text = re.sub('\s\W',' ',text)
    text = re.sub('\W,\s',' ',text)
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub("\d+", "", text)
    text = re.sub('\s+',' ',text)
    text = re.sub('[!@#$_]', '', text)
    text = text.replace("co","")
    text = text.replace("https","")
    text = text.replace(",","")
    text = text.replace("[\w*"," ")
    
    return text
df['question1'] = [cleaning(text) for text in df['question1']]
df['question2'] = [cleaning(text) for text in df['question2']]

In [ ]:
newdf=df.drop(['qid1','qid2'], axis=1)


In [ ]:
newdf.head()

*Jaccard Similarity*

In [ ]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [ ]:
val=0
count=0
for i in range(20000):
    sentences = [newdf['question1'][i],
    newdf['question2'][i]]
    sentences = [sent.lower().split(" ") for sent in sentences]
    if jaccard_similarity(sentences[0], sentences[1])<0.5:
      ans=True
    else:
      ans=False
    if ans==newdf['is_duplicate'][i]:
      count+=1
print("accuracy=",count/20000)

accuracy=34.6%

*Levenshtein Distance*

In [ ]:
count=0
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        s1, s2 = s2, s1

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]/float(len(s1))
for i in range(10000):
  s1 = newdf['question1'][i]
  s2 = newdf['question2'][i]
  example_1 = (s1, s2)
  if levenshtein(*example_1)<0.5:
      ans=True
  else:
      ans=False
  if ans==newdf['is_duplicate'][i]:
    count+=1
print("accuracy=",count/10000)

accuracy=66.5%

*Cosine Similarity*

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
count=0
# Program to measure the similarity between
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# X = input("Enter first string: ").lower()
# Y = input("Enter second string: ").lower()



for i in range(50000):
  X = newdf['question1'][i]
  Y = newdf['question2'][i]

  # tokenization
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)

  # sw contains the list of stopwords
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form a set containing keywords of both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0

  # cosine formula
  for i in range(len(rvector)):
      c+= l1[i]*l2[i]
  if float((sum(l1)*sum(l2))**0.5)!=0:
    cosine = c / float((sum(l1)*sum(l2))**0.5)
  else:
    continue;
  if cosine<0.5:
      ans=True
  else:
      ans=False
  if ans==newdf['is_duplicate'][i]:
    count+=1
print(count)
print("accuracy=",count/50000)

accuracy=58.3%

LSTM


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
test_data = df


In [ ]:
X_train = df.iloc[:,:5].values
Y_train = df.iloc[:,5:].values
X_testq1 = test_data.iloc[:400001,1:2].values
X_testq2 = test_data.iloc[:400001, 2:].values
s1 = X_train[:,3]
s2 = X_train[:,4]

In [ ]:
def tokenize(s):
    tokens = []
    tokens = [word_tokenize(str(sentence)) for sentence in s]

    rm1 = []
    for w in tokens:
        sm = re.sub('[^A-Za-z]',' ', str(w))
        x = re.split("\s", sm)
        rm1.append(x)
        
    return rm1

In [ ]:
def lower_case(s):
    #Removing whitespaces    
    for sent in s:
        while '' in sent:
            sent.remove('')

    # Lowercasing
    low = []
    for i in s:
        i = [x.lower() for x in i]
        low.append(i)
        
    return low

In [ ]:
def lemmatize(s):
    lemma = []
    wnl = WordNetLemmatizer()
    for doc in s:
        tokens = [wnl.lemmatize(w) for w in doc]
        lemma.append(tokens)

    # Removing Stopwords
    filter_words = []
    Stopwords = set(stopwords.words('english'))

    #ab = spell('nd')
    for sent in lemma:
        tokens = [w for w in sent if w not in Stopwords]
        filter_words.append(tokens)

    space = ' ' 
    sentences = []
    for sentence in filter_words:
        sentences.append(space.join(sentence))
        
    return sentences

In [ ]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [ ]:
# X_train_q1 = tokenizer.texts_to_sequences(np.array(listq1))
X_train_q1 = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_train_q2 = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')

In [ ]:
X_test_q1 = tokenizer.texts_to_sequences(X_test_q1.ravel())
X_test_q1 = pad_sequences(X_test_q1,maxlen = 30, padding='post')

X_test_q2 = tokenizer.texts_to_sequences(X_test_q2.astype(str).ravel())
X_test_q2 = pad_sequences(X_test_q2, maxlen = 30, padding='post')


In [ ]:
word_index = tokenizer.word_index

In [ ]:
embedding_index = {}
with open('../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Transformer Based Semantic Similarity


In [ ]:
!pip install transformers

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
count=0
for i in range(10000):
  sentences = [newdf['question1'][i],
    newdf['question2'][i]]
  sentence1 = sentences[0]
  sentence2 = sentences[1]
  # encode sentences to get their embeddings
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  if cosine_scores.item()>0.5:
    t=1
  else:
    t=0
  if t==newdf['is_duplicate'][i]:
      count+=1
  print(i)
print("accuracy=",count/10000)

accuracy after 10000 sentences= 60.31%

WMD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyemd

In [ ]:
import gensim
from gensim.models import Word2Vec
    
model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/testing/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
from pyemd import emd
from gensim.similarities import WmdSimilarity
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedLineDocument

In [ ]:
global PYEMD_EXT
try:
    from pyemd import emd
    PYEMD_EXT = True
except ImportError:
    PYEMD_EXT = False

In [ ]:
question3 = 'Why am I mentally very lonely? How can I solve it?'
question4 = 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?'
question3 = question3.lower().split()
question4 = question4.lower().split()
distance = model.WmdSimilarity(question3, question4)
print('distance = %.4f' % distance)

In [ ]:
model=Word2Vec(sentences=common_texts)
for i in range(10000):
  
  sentences = [newdf['question1'][i],
    newdf['question2'][i]]
  
  sentence1 = sentences[0].lower().split()
  sentence2 = sentences[1].lower().split()
  
  distance= model.wmdistance(sentence1, sentence2)